In [1]:
import networkx as nx
from typing import List, Tuple, Dict
import random

from routing_game_utils import *

In [3]:
def mixed(
    G: nx.DiGraph,
    source_sink_pairs: List[Tuple[int, int]],
    latency_functions: Dict,
    max_iterations: int = 1000,
    learning_rate: float = .1,
    marginal_cost_pricing: bool = False,
    sensitivities: List[float] = None,
    verbose: bool = False
):
    # each player's strategy set is represented as a set of available paths from their source to their sink
    paths = get_strategy_sets(G, source_sink_pairs)
    num_players = len(source_sink_pairs)

    # start with no bias for any particular strategy
    logits = [np.array([1.0 for _ in range(len(player_path))]) for player_path in paths]

    # for each player, initialize to random strategy
    random.seed(10)
    current_flow = [random.randint(0, len(paths[player_idx]) - 1) for player_idx in range(num_players)]

    if verbose:
        print('paths ', paths)
        print('starting flow ', current_flow)
        
    iteration = 0

    # debugging information, to make sure players are picked uniformly at random
    picked_players = [0 for _ in range(num_players)]


    historical_avg = [0 for _ in range(num_players)]
    
    while iteration < max_iterations:
        
        if verbose:
            print('-' * 8, 'starting round: ', iteration, '-' * 8)
        iteration += 1

        # previous flows, latencies

        # TODO: can we reduce the amount of times this calculation is called so it's more efficient?
        if (marginal_cost_pricing):
            f_e, player_latencies, total_latency = calculate_edge_flows_taxed(paths, current_flow, latency_functions, sensitivities)
        else:
            f_e, player_latencies, total_latency = calculate_edge_flows(paths, current_flow, latency_functions)
        player_idx = random.randint(0, num_players - 1) # pick a random player
        picked_players[player_idx] += 1

        # get previous latency/path
        prev_latency = player_latencies[player_idx]
        prev_path_idx = current_flow[player_idx]
        prev_path = paths[player_idx][prev_path_idx]

        alpha = 0.2
        prev_utility = -1 * prev_latency

        # update the historical average
        historical_avg[player_idx] = (1 - alpha) * historical_avg[player_idx] + alpha * prev_utility
        
        # construct a new flow
        mixed_strat = softmax(logits[player_idx]) # get a player's mixed strategy from their logits

        # even if we were at the best move, and we pick a worse alternative
        # what would happen is that we would evaluate the alternative to be bad, lower its probability of happening
        # then maybe go back to the best, which by virtue of being the best, we would increase its probability of happening again
        # this way, good moves get rewarded, bad moves get punished probabilistically
        alternative_path_idx = sample_move_skip_index(mixed_strat, prev_path_idx) # pick some alternate move (no point of repeating current move)
        alternative_path = paths[player_idx][alternative_path_idx]
        if sensitivities:
            s_i = sensitivities[player_idx]
        else:
            s_i = 1

        # debugging information
        if verbose:
            print('picking player', player_idx)
            print('previous latency', prev_latency)
            print('prev path idx', prev_path_idx)
            print('alternative path idx', alternative_path_idx)
            print('alternative_path')

        # develop alternative path
        hypothetical_f_e = f_e.copy()
        demand = 1 # treat each player as controlling 1 unit of demand

        # constructing the alternative flow
        # step 1: Remove player's demand from the path's edges they are currently using
        for i in range(len(prev_path) - 1):
            edge = (prev_path[i], prev_path[i + 1])
            hypothetical_f_e[edge] -= demand

        # step 2: Add player's demand to alternative path edges
        for i in range(len(alternative_path) - 1):
            edge = (alternative_path[i], alternative_path[i + 1])
            hypothetical_f_e[edge] = hypothetical_f_e.get(edge, 0) + demand

        # step 3: Compute latency for the alternative path
        alternative_latency = 0
        for i in range(len(alternative_path) - 1):
            edge = (alternative_path[i], alternative_path[i + 1])
            congestion = hypothetical_f_e[edge]
            f, f_prime = latency_functions[edge]
            if marginal_cost_pricing:
                latency = f(congestion)
                tax = congestion * f_prime(congestion)
                edge_latency =  latency + s_i * tax
            else:
                edge_latency = f(congestion)
            alternative_latency += edge_latency
        
        alternative_util = -1 * alternative_latency

        # get difference between alternative utility and current utility
        # utility is -1 * latency
        # give logit a positive increase for good delta where alternative utility increased
        delta = alternative_util - historical_avg[player_idx]

        # debugging data
        if verbose:
            print('delta', delta)
            print('old logits: ', logits[player_idx])

        # update logits. positive delta = alternative was good, and should be picked more frequently in the future
        logits[player_idx][alternative_path_idx] += learning_rate * delta

        # debugging data
        if verbose:
            print('new logits: ', logits[player_idx])

        current_flow[player_idx] = alternative_path_idx

    # get final mixed strategy profile
    probabilities_profile = [softmax(player_logits).tolist() for player_logits in logits]
    # print(picked_players)
    # paths: strategy sets for each player
    # current_flow: the equilibrium flow. each idx represents a player, each value represents the strategy that the player picked (ex: strategy 0, 1, ...)
    # player_latencies: the latency of each player in the equilibrium flow
    # total latency: sum of player latencies
    return paths, probabilities_profile

In [4]:
def argmax_flow(probabilities_profile):
    flow = []
    for mixed in probabilities_profile:
        flow.append(int(np.argmax(mixed)))
    return flow

In [5]:
def test(filename, marginal_cost_pricing=False):
    G, source_sink_pairs, latency_functions = json_to_atomic_instance(filename)
    paths, probabilties_profile = mixed(G, source_sink_pairs, latency_functions, 20000, .02, marginal_cost_pricing)
    flow = argmax_flow(probabilties_profile)

    f_e, player_latencies, total_latency = calculate_edge_flows(paths, flow, latency_functions)
    
    if (marginal_cost_pricing):
        f_e_taxed, player_latencies_taxed, total_latency_taxed = calculate_edge_flows_taxed(paths, flow, latency_functions)

        print('total latency taxed: ', total_latency_taxed)
    
    print('total latency untaxed: ', total_latency)

    print('All paths: ', paths)

    print('Probabilities profile: ', probabilties_profile)
    print('argmax flow: ', flow)


In [12]:
test('graph_examples/braess.txt')

total latency untaxed:  152
All paths:  [[['s', 'v', 't'], ['s', 'v', 'w', 't'], ['s', 'w', 't']], [['s', 'v', 't'], ['s', 'v', 'w', 't'], ['s', 'w', 't']], [['s', 'v', 't'], ['s', 'v', 'w', 't'], ['s', 'w', 't']], [['s', 'v', 't'], ['s', 'v', 'w', 't'], ['s', 'w', 't']], [['s', 'v', 't'], ['s', 'v', 'w', 't'], ['s', 'w', 't']], [['s', 'v', 't'], ['s', 'v', 'w', 't'], ['s', 'w', 't']], [['s', 'v', 't'], ['s', 'v', 'w', 't'], ['s', 'w', 't']], [['s', 'v', 't'], ['s', 'v', 'w', 't'], ['s', 'w', 't']], [['s', 'v', 't'], ['s', 'v', 'w', 't'], ['s', 'w', 't']], [['s', 'v', 't'], ['s', 'v', 'w', 't'], ['s', 'w', 't']]]
Probabilities profile:  [[0.6057955284316281, 0.0018583537460113437, 0.3923461178223605], [0.12123602979576782, 0.0011893867301064763, 0.8775745834741258], [0.06462330725670865, 0.0009832435251741322, 0.9343934492181172], [0.633265656752007, 0.0005765868588548482, 0.3661577563891382], [0.16508279424624328, 0.0012755171658195724, 0.8336416885879372], [0.186010400786823, 0.00049